# Forest carbon stock from look-up tables
This notebook reads, transforms, and plots selected elements of the look-up tables from Schedule 6 of the Climate Change (Forestry Sector) Regulations 2008.

The look-up tables are stored as four separate files in CSV format, e.g. ``./mpi-tables/table1.csv``. They contain pre-calculated values for the *cumulative* carbon stock $S_a^T$ measured in tonnes per hectare of a given forest age $a=0,1,\dots,50$ (in years) and forest type $T$. We can calculate the annual carbon stock *change* $s_a^T$ using the following formula,

$$
s_a^T = \left\{
\begin{array}{lcl}
0 & \mathrm{for} & a = 0, \\
S_a^T - S_{a-1}^T & \mathrm{for} & a = 1,\dots, 50.
\end{array} \right.
$$

The graphs below plot $S_{a}^{T}$ and $s_{a}^{T}$ vs $a$ for a given $T$, which the user can select using a drop-down menu.

In [1]:
import pandas as pd             # for handling dataframes
import matplotlib.pyplot as plt # for plotting data
import ipywidgets as widgets    # for interactivity

In [2]:
# Core function for calculating annual carbon stock change per hectare
def stock_change(y, T, table):
    return table[T][y] - table[T][y-1]

# Function for calling stock_change over the entire time-span 
def stock_changes(T, table):
    values=[0]
    for y in range(1, len(table)):
        values.append(stock_change(y,T,table))
    return values

# load first two tables as dataframes and merge into one
table = pd.concat([pd.read_csv('table1.csv'),
                   pd.read_csv('table2.csv')], axis=1)

# drop both 'Age' columns, since age matches row index
table.drop('Age (yrs)', inplace=True, axis=1)

# Prepend "P.Radiata" to type names from 
subs = {}
for label in table.columns:
    if(len(label.split()) == 1): 
        subs[label]="P.Radiata in "+label
table = table.rename(columns = subs)

# show first few rows of the final table
table.head(5)

,P.Radiata in Ak,P.Radiata in W/T,P.Radiata in BOP,P.Radiata in Gis,P.Radiata in H/SNI,P.Radiata in N/M,P.Radiata in C/W,P.Radiata in O,P.Radiata in S,Douglas fir,Exotic softwoods,Exotic hardwoods,Indigenous forest
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.5,0.4,0.4,0.6,0.5,0.2,0.2,0.3,0.2,0.1,0.2,0.1,0.6
2,3.0,3.0,2.0,4.0,3.0,1.0,1.0,2.0,1.0,0.1,1.0,3.0,1.2
3,8.0,7.0,6.0,10.0,9.0,3.0,2.0,5.0,3.0,0.4,3.0,13.0,2.5
4,29.0,25.0,24.0,37.0,34.0,12.0,5.0,9.0,14.0,1.0,12.0,34.0,4.6


In [3]:
# Wrap the plotting in a function for future use
def create_plot(T, table):
    
    years = list(range(len(table)))
    stock = list(table[T].values)
    change = stock_changes(T=T,table=table);

    # Instantiate figure for plotting
    fig, axs = plt.subplots(2, figsize=[8.0,6.0], sharex = True)
    
    axs[0].plot(stock)
    axs[0].set_ylim(0)
    axs[0].set_ylabel("carbon stock \n (tons/ha)")
    
    axs[1].bar(x = years, 
               height = change, 
               width = -1.0, # -ve to align with right edge
               align = 'edge')
    axs[1].set_xlim(years[0],years[-1])
    axs[1].set_xlabel("age (years)")
    axs[1].set_ylabel("annual change \n(tons/ha)")

    return fig

#fig = create_plot(T = "Indigenous forest", table = table)

# Define widgets for the forest type
T_widget = widgets.Dropdown(options = list(table.columns), 
                            value = 'Indigenous forest', 
                            description='forest type')

# Instantiate an interactive plot
ifig = widgets.interactive(create_plot, 
                           T = T_widget, 
                           table = widgets.fixed(table))

# Show the plot
ifig

interactive(children=(Dropdown(description='forest type', index=12, options=('P.Radiata in Ak', 'P.Radiata in …